C++ presentation of 'Templet application' interface and model
=============================================================

```
 ------------------------------------------------------------
             Commands           |                Roles
 ------------------------------------------------------------
  REST commands |  CLI commands | RESOURCE | OBSERVER | USER 
 ---------------|---------------|----------------------------
 write_event    |  event        |    +     |          | 
 read_events    |  log          |    +     |    +     | 
 reply_on_query |  reply        |    +     |          | 
 write_query    |  query        |          |          |   +
 read_answer    |  answer       |          |    +     |   

SESSION(NAME,RESOURCE,OBSERVER,USER)
EVENT(ORDINAL,TAG,GUID,DATA,NAME)
ANSWER(GUID,DATA,NAME)

write_event(SESSION,TAG,DATA)        --> ORDINAL \/ false
read_events(SESSION,ORDINAL)         --> EVENT[] \/ false
write_query(SESSION,GUID,TAG,DATA)   --> true \/ false
reply_on_query(SESSION,ORDINAL,DATA) --> true \/ false
read_answer(SESSION,GUID)            --> DATA \/ false
```

In [4]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // service interface    

typedef string   NAME;
typedef unsigned ORDINAL;
typedef unsigned TAG;
typedef unsigned GUID;

#define ASSERT(expr)  if(!(expr))return false;
    
struct SESSION{
    NAME name;      // who is the session owner?
    bool RESOURCE;// what access does the session owner have?
    bool OBSERVER;
    bool USER;
}; 

struct DATA{ long size; void* data; }; //abstract binary data

struct EVENT{
    ORDINAL ord;
    TAG     tag;
    GUID    guid;
    DATA    data;
    NAME    name;
};

struct ANSWER{
    GUID    guid;
    NAME    name; // who can read the answer?
    DATA    data;
};

/*-1-*/
bool write_event(SESSION s,TAG t,DATA d,ORDINAL& ord){
   
    ASSERT(s.RESOURCE);
    
    EVENT ev;
    
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.guid = 0; // no guid for regular event
    ev.data = d; ev.name = s.name;
    
    events.push_back(ev);
    
    return true;
}
    
/*-2-*/              
bool read_events(SESSION s,ORDINAL ord,list<EVENT>& evs){
    
    ASSERT(s.RESOURCE || s.OBSERVER);

    evs.clear();
    
    for(EVENT ev:events){
        if(ev.ord>=ord) evs.push_back(ev);
    }   
    return true;
}

/*-3-*/
bool write_query(SESSION s,GUID guid,TAG t,DATA d){
    
    ASSERT(s.USER && guid!=0);
    
    EVENT ev;
    
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.guid = guid; 
    ev.data = d; ev.name = s.name;
    
    events.push_back(ev);
    
    return true;
}

/*-4-*/    
bool reply_on_query(SESSION s,ORDINAL ord,DATA d){

    ASSERT(s.RESOURCE);

    for(EVENT ev:events){
        if(ev.ord==ord && ev.guid!=0){
            ANSWER ans;
            ans.guid = ev.guid; ans.name = ev.name; ans.data = d;
            answers.push_back(ans);
            return true;
        }
    }
    return false;
}    

/*-5-*/            
bool read_answer(SESSION s,GUID guid,DATA& d){
   
    ASSERT(s.USER || s.OBSERVER);

    for(ANSWER ans:answers){
        if(ans.guid == guid){
            ASSERT(s.USER ? s.name==ans.name : s.OBSERVER);
            d = ans.data;
            return true;
        }
    }   
    return false;
}

private: // service state
    
list<EVENT>  events;
list<ANSWER> answers;

};

```
Интерфейс параметров командной строки
=====================================

Идея: возможно реализовать сервис синхронизации глобального состояния на базе готовых систем удаленного запуска задач (Everest, HTCondor, Dirac и т.д.) для быстрого прототипирования и использования в качестве эталонной реализации для полнофункционального сервиса, а затем CLI-интерфейса полнофункционального сервиса.
Для этого реализуется консольное приложение с описанным ниже интерфейсом.

Параметры
=========
(--base | -b) <base> база данных состояния приложения
(--tag  | -t) <num> тэг события
(--ordinal | -о) <num> порядковый номер события
(--answer  | -a) <file> ответ на запрос
(--guid | -g) <str> уникальный идентификатор запроса 

Форматы файлов и stdout
=======================
<file>     формат определяется в приложении
<base>     SQLite файл базы данных, хранящей состояние приложения в виде последовательности событий его обновления
<logfile>  запись фрагмента последовательных событий обновления состояния, внутренний формат

Команды
=======
(--event | -e) <file>  запись события, параметры: base, tag, в stdout - порядковый номер события
(--log   | -l) <logfile> выборка из лога событий, начиная с заданного ordinal, параметры: base, ordinal
(--reply | -r) <file>  ответ на запрос с указанным номером ordinal, параметры: base, ordinal
(--query | -q) <file>  файл с данными запроса, параметры: base, tag, guid, answer (если указан, выполняется ожидание ответа)
(--answer| -a) <file>  файл с ответом на запрос (если не указан query), параметры: base, guid  
```